In [1]:
%load_ext autoreload
%autoreload 2

import os
if not os.getcwd().endswith('BENDR'): os.chdir(os.path.dirname(os.getcwd()))
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
from tqdm import tqdm
from pathlib import Path

import BENDR.utils as utils
from importlib import reload
import BENDR.BENDR_utils as BENDR_utils
reload(BENDR_utils)
from BENDR.BENDR_utils import LinearBENDR, LinearHeadBENDR
from matplotlib import pyplot as plt
import activation_generator as act_gen
import numpy as np
import tcav as tcav
import tensorflow as tf
import utils_plot
import utils as tcav_utils
import pickle

2023-04-25 14:03:45.456148: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 14:03:46.743434: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Results

In [2]:
import pickle
from scipy.stats import ttest_ind

with open('tcav_results_all.pkl', 'rb') as handle:
    results = pickle.load(handle)
    
random_counterpart = None
random_concepts = None 
num_random_exp = 5
min_p_val = 0.05

In [3]:
results

[{'cav_key': 'fake-random500_0-summarizer-linear-0.1',
  'cav_concept': 'fake',
  'negative_concept': 'random500_0',
  'target_class': 'left',
  'cav_accuracies': {'fake': 0.625,
   'random500_0': 0.2222222222222222,
   'overall': 0.4117647058823529},
  'i_up': 0.24,
  'val_directional_dirs_abs_mean': 0.408141334373699,
  'val_directional_dirs_mean': 0.27970343662115454,
  'val_directional_dirs_std': 0.4021373832796861,
  'val_directional_dirs': [0.4079159145043164,
   0.6639458837003445,
   0.3927444520698466,
   -0.34380036015946003,
   1.0693037372017609,
   0.23453506145513536,
   0.6940615614624654,
   0.3640708461522455,
   0.3835564770155328,
   0.2659284534729005,
   -0.0009330017951924008,
   -0.3185511441423359,
   0.6413490019691086,
   0.426576690290115,
   0.2980031153754663,
   0.3060863217432688,
   0.21665301935515024,
   -0.2178138854587638,
   0.38025531030520376,
   0.19099286066251608,
   -0.3920367248097356,
   0.10954237106622011,
   -0.33233860554131844,
   0.299

In [4]:
summary = tcav_utils.print_results(results, num_random_exp=2)

Class = left
  Concept = fake
    Bottleneck = summarizer. TCAV Score = 0.62 (+- 0.41), random was 0.44 (+- 0.42). p-val = 0.217 (not significant)


# TCAV RUN 

In [2]:
folder_path = Path('/mnt/c/Users/anders/OneDriveDTU/Dokumenter/BENDR')

encoder_weights = folder_path / 'encoder_BENDR_linear_2_1024_20.pt'
enc_augment_weights = folder_path / 'enc_augment_BENDR_linear_2_1024_20.pt'
classifier_weights = folder_path / 'classifier_BENDR_linear_2_1024_20.pt'
extended_classifier_weights = folder_path / 'extended_classifier_BENDR_linear_2_1024_20.pt'

model = LinearBENDR(targets=2, samples=1024, channels=20, device='cpu')
model.load_all(encoder_weights, enc_augment_weights, classifier_weights, extended_classifier_weights)
model = model.train(False)

Receptive field: 143 samples | Downsampled by 96 | Overlap of 47 samples | 10 encoded samples/trial | 4 pooled


In [3]:
filename = 'data/concepts/fake/aaaaaarq_s004_t00127.pkl'
with open(filename, "rb") as f:
    concept = pickle.load(f).to(torch.device('cpu'))

In [ ]:
# def normalize(arr):
#     tmp = arr[:, :19, :]
#     tmp = 2 * (tmp - tmp.min()) / (tmp.max() - tmp.min()) - 1
#     arr[:, :19, :] = tmp
#     return arr

# path = 'data/concepts/'
# import glob
# # Go through each 1x20x1024 tensor in the path folder and subfolders and normalize it between -1 and 1 and overwrite the file.

# files = []
# for file in glob.glob(path + '**/*.pkl', recursive=True):
#     if "/random" not in file and "eyes/" not in file and "musc/" not in file:
#         files.append(file)

# for filename in tqdm(files):
#     with open(filename, "rb") as f:
#         concept = pickle.load(f)
#     concept = normalize(concept)
#     with open(filename, "wb") as f:
#         pickle.dump(concept, f)

In [4]:
source_dir = 'data/concepts'
results_dir =  'data/tcav_results' 
activation_dir = 'data/activations'
cav_dir = 'data/cavs'
bottlenecks = ['summarizer'] #['extended_classifier', 'summarizer', 'classifier']
alphas = [0.1]

target =  'left' #'Right fist, performed'

# concepts are stored in folders with these names
concepts =  ['motor_lh', 'motor_rh'] #['eyem', 'musc'] #['spsw', 'artf', 'bckg', 'gped', 'pled', 'eyem', 'musc']

labels = ['left', 'right']

In [5]:
from model import EEGWrapper, BENDR_cutted

class BENDRWrapper(EEGWrapper) : 
    def __init__(self, model, labels, sample_length_target):
        eeg_shape = [1, 20, sample_length_target]
        super(BENDRWrapper, self).__init__(eeg_shape=eeg_shape, eeg_labels=labels)
        self.model = model
        self.model_name = 'BENDR'

    def forward(self, x):
        return self.model.forward(x)

    def get_cutted_model(self, bottleneck):
        return BENDR_cutted(self.model, bottleneck)
    
tcav_model = BENDRWrapper(model, labels, 1024)

In [12]:
act_generator = act_gen.EEGActivationGenerator(
   tcav_model, source_dir, activation_dir, max_examples=25
   )

In [13]:
tf.compat.v1.logging.set_verbosity(0)
num_random_exp = 2

my_tcav = tcav.TCAV(target,
                   concepts,
                   bottlenecks,
                   act_generator,
                   alphas,
                   cav_dir=cav_dir,
                   num_random_exp=num_random_exp)

print('Loading mytcav')
results = my_tcav.run()

Loading mytcav
Running param 0 of 6
Running param 1 of 6
Running param 2 of 6
Running param 3 of 6


KeyboardInterrupt: 

: 

In [87]:
acts = act_generator.process_and_load_activations(
            ['extended_classifier'], ['fake'] + ['left'])

In [91]:
acts['left']['extended_classifier']

array([[ 1.7540379 , -0.82603973],
       [ 1.8674215 , -1.3207444 ],
       [ 2.500525  , -2.9040377 ],
       [ 0.03337348, -0.02071789],
       [ 0.13110691, -0.96623176],
       [ 1.8097417 , -1.0863051 ],
       [ 1.3367037 , -1.5014396 ],
       [ 1.0848595 , -0.8973087 ],
       [ 1.8659656 , -1.5945015 ],
       [ 1.4206934 , -1.4964838 ],
       [ 0.2003327 ,  0.5273163 ],
       [ 1.9878415 , -1.9304913 ],
       [-0.05725542,  1.0587088 ],
       [-0.45656103,  1.1971372 ],
       [ 0.28905174,  0.25801787],
       [ 1.4830573 , -1.2059613 ],
       [ 3.0847242 , -1.6473688 ],
       [ 2.5295148 , -2.8055615 ],
       [ 0.45311543, -0.90106064],
       [ 1.6675662 , -1.0756153 ],
       [ 2.458944  , -2.2656436 ],
       [ 1.7312605 , -1.258964  ],
       [ 1.3342994 , -0.41952518],
       [ 0.46867287, -0.31197387],
       [-1.0063612 ,  1.4782271 ]], dtype=float32)

In [79]:
act_generator.get_activations_for_concept('left', 'classifier')

array([[-1.186353  ,  1.4381859 ],
       [ 0.4448976 , -0.5412097 ],
       [ 2.52803   , -1.8295906 ],
       [ 2.500525  , -2.9040377 ],
       [ 1.4206934 , -1.4964838 ],
       [ 1.5175933 , -1.1672606 ],
       [ 2.042131  , -1.0454305 ],
       [ 0.7111686 , -0.04078286],
       [ 1.8082571 , -1.3435547 ],
       [ 1.1702008 , -1.3657184 ],
       [ 0.6762678 , -0.9201929 ],
       [ 1.1328269 , -0.4346566 ],
       [-0.7604427 ,  0.5122205 ],
       [ 0.29859874,  0.22791338],
       [ 2.3852735 , -3.0166943 ],
       [ 1.1235342 , -0.56455475],
       [ 2.5295148 , -2.8055615 ],
       [ 0.8691017 , -0.16881815],
       [ 1.6816764 , -1.1845719 ],
       [ 2.0413084 , -1.3851118 ],
       [ 0.63697827, -0.38213032],
       [ 0.4352816 , -0.04616177],
       [ 0.33295324,  0.05565448],
       [ 1.6516477 , -1.4725301 ],
       [ 0.69876236, -0.796599  ]], dtype=float32)

In [50]:
for result in results:
    print(result['cav_concept'])

Dorsal Stream Visual Cortex-lh
Dorsal Stream Visual Cortex-lh
DorsoLateral Prefrontal Cortex-rh
DorsoLateral Prefrontal Cortex-rh
random500_0
random500_1
